## Notebook demonstrating the addition of data segmented with [proseg](https://github.com/dcjones/proseg)

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from insitupy import InSituData, CACHE

## Load data

In [3]:
insitupy_project = Path(CACHE / "out/demo_insitupy_project")
xd = InSituData.read(insitupy_project)
xd.load_all()

In [4]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\.cache\InSituPy\out\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       MultiCellData with main layer 'main'
           matrix
               AnnData object with n_obs × n_vars = 167780 × 313
               obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
               var: 'gene_ids', 'feature_types', 'genome'
               obsm: 'spatial'
           boundaries
               BoundariesData object with 2 entries:
                   cells
                   nuclei
    ➤ transcripts
       DataFrame with shape Delayed('int-221870c4-7ac9-459b-8d4c-f2ed90c84124') x 8

## Select small region for demonstration

In [5]:
xdcrop = xd.crop(xlim=(2700,3000), ylim=(2700,3000))

## Export transcripts for proseg

In [6]:
transcripts_out_path = Path(CACHE / "out/transcripts_for_proseg.csv")
transcripts_out_path.parent.mkdir(exist_ok=True)

In [7]:
# export transcripts as csv
xdcrop.transcripts.to_csv(transcripts_out_path, single_file=True)

['C:\\Users\\ge37voy\\.cache\\InSituPy\\out\\transcripts_for_proseg.csv']

## Install proseg

For installation checkout the installation instructions in the [proseg Github repository](https://github.com/dcjones/proseg?tab=readme-ov-file#installing). In brief, proseg is a [Rust](https://www.rust-lang.org/) package and can be installed using:

```Bash
cargo install proseg
```

## Run proseg

```Bash
proseg /path/to/transcripts.csv.gz
```

In [7]:
output_path = transcripts_out_path.parent / "proseg_results"
output_path.mkdir(exist_ok=True)

In [9]:
import subprocess

# Start the process
process = subprocess.Popen([
    'proseg',
    '--xenium', str(transcripts_out_path),
    '--output-path', str(output_path)
    ], stdout=subprocess.PIPE)

# Continuously read the output
while True:
    output = process.stdout.readline()
    if output == b'' and process.poll() is not None:
        break
    if output:
        print(output.decode('utf-8', errors='replace').strip())

Using 16 threads
Read 144377 transcripts
590 cells
471 genes
Estimated full area: 90017.61
Full volume: 549952.6
Using grid size 123.504875. Chunks: 9


## Add proseg results to `InSituData`

In [8]:
xdcrop.cells.add_proseg(path=output_path)
xdcrop.cells.add_proseg(path=output_path, key="test") # add the data a second time with another key

In [9]:
cropped_out = CACHE / "out/cropped"
xdcrop.saveas(cropped_out)

Saving data to C:\Users\ge37voy\.cache\InSituPy\out\cropped
Saved.


## Store and visualize data

In [10]:
xdr = InSituData.read(cropped_out)
xdr.load_all()

In [ ]:
# visualize data
xdr.show()

Layer 'proseg-boundaries-cells' already in layer list.
